In [31]:
from statsbombpy import sb
import math
import pandas as pd
from pandasql import sqldf

In [2]:
columns = [
    'id',
    'match_id',
    'index',
    'minute',
    'second',
    'possession',
    'possession_team_id',
    'possession_team',
    'type',
    'player_id',
    'player',
    'position',
    'team_id',
    'team',
    'location',
    'pass_end_location',
    'pass_outcome',
    'ball_receipt_outcome',
    'pass_recipient_id',
    'pass_recipient',
    'carry_end_location',
    'dribble_outcome',
    'foul_won_advantage',
    'related_events',
    'under_pressure'
]

events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2015/2016",
    gender="male"
)[columns]

events = events[events.match_id.isin([3890305, 3890561, 3890511, 3890515, 3890263, 3890261])]

events.head()

/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data acce

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,pass_end_location,pass_outcome,ball_receipt_outcome,pass_recipient_id,pass_recipient,carry_end_location,dribble_outcome,foul_won_advantage,related_events,under_pressure
0,41bd60ac-9b2c-4cb8-85aa-23ae71825c1e,3890561,1,0,0,1,175,Hoffenheim,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fbca533d-f3f4-4a86-b4a3-4fcae63592cf,3890561,2,0,0,1,175,Hoffenheim,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20ae06cd-9bf8-4797-a14a-2ad74773e772,3890511,1,0,0,1,173,Hertha Berlin,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,e27699cf-9ddb-41b5-a408-e0e682931688,3890511,2,0,0,1,173,Hertha Berlin,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,d4d15ebe-fa14-41a5-94cd-febba3555581,3890515,1,0,0,1,171,Hamburger SV,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
matches = sb.matches(competition_id=9, season_id=27)[['match_id', 'competition', 'season', 'match_date', 'home_team', 'home_score', 'away_score', 'away_team']]

matches = matches[matches.match_id.isin([3890305, 3890561, 3890511, 3890515, 3890263, 3890261])]

matches

/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,competition,season,match_date,home_team,home_score,away_score,away_team
0,3890561,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
2,3890511,Germany - 1. Bundesliga,2015/2016,2016-04-08,Hertha Berlin,2,2,Hannover 96
3,3890515,Germany - 1. Bundesliga,2015/2016,2016-04-09,Hamburger SV,1,2,Darmstadt 98
263,3890305,Germany - 1. Bundesliga,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg
301,3890263,Germany - 1. Bundesliga,2015/2016,2015-08-15,FSV Mainz 05,0,1,Ingolstadt
303,3890261,Germany - 1. Bundesliga,2015/2016,2015-08-15,Borussia Dortmund,4,0,Borussia Mönchengladbach


In [4]:
def pass_start_condition(x):
    return events[events.id == list(filter(lambda x: events[events.id == x].type.item() == 'Pass', x))[0]].location.item()[0] < 60

In [5]:
ball_receipts = events[(events.type == 'Ball Receipt*') & (events.ball_receipt_outcome.isna()) & (events.position.str.contains('Midfield'))]
ball_receipts = ball_receipts[ball_receipts.location.apply(lambda x: 20 < x[0] < 60)]
ball_receipts = ball_receipts[ball_receipts.related_events.apply(pass_start_condition)]
ball_receipts = pd.merge(left = ball_receipts, right = matches, how = 'left', on = 'match_id')

ball_receipts.head()

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,foul_won_advantage,related_events,under_pressure,competition,season,match_date,home_team,home_score,away_score,away_team
0,69e3b0a7-5847-40db-91d5-0e24fb150f63,3890561,9,0,2,2,175,Hoffenheim,Ball Receipt*,6039.0,...,NaN,[77e2ddaf-6de3-49e7-a318-7d765799b543],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
1,d843ba4b-de1e-4e81-a71d-0f3781c8f3fa,3890561,63,0,56,3,181,Schalke 04,Ball Receipt*,3053.0,...,NaN,[57acd29e-7ac3-45e9-acaa-cfa96d74c10f],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
2,45271883-07e0-446f-ba83-3ddc3e678def,3890561,69,1,4,3,181,Schalke 04,Ball Receipt*,5242.0,...,NaN,[e5126f2f-a414-4954-a607-927f102fe5bb],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
3,7d08b692-1f2e-4b44-be64-7d3e04632ab9,3890561,86,1,14,3,181,Schalke 04,Ball Receipt*,5242.0,...,NaN,[cbeb8953-feeb-45d0-84ce-a0416be4eeba],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
4,0a325d10-463e-4e70-86bd-1d2ae118864c,3890561,92,1,19,3,181,Schalke 04,Ball Receipt*,5242.0,...,NaN,[7bb92ebe-6834-4cdf-9888-5dc19f0a53e3],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04


In [6]:
data = dict()
for receipt in ball_receipts.itertuples():
    event_dict = dict()
    event_count = 0
    final_location_x = receipt.location[0]
    players = set()
    dribble = False
    last_carry_start_location = receipt.location
    last_carry_distance = 0
    possession_events = events[(events.match_id == receipt.match_id) & (events.possession == receipt.possession) & (events['index'] > receipt.index)].sort_values('index')
    for event in possession_events.itertuples():
        if final_location_x >= 80 or event_count == 4 or len(players) == 2:
            break
        else:
            if event.type == 'Pass' and pd.isna(event.pass_outcome):
                event_count += 1
                final_location_x = event.pass_end_location[0]
                players.add(event.player_id)
            elif dribble == False and event.type == 'Carry':
                if math.dist(event.location, event.carry_end_location) > 5.46807:
                    event_count += 1
                final_location_x = event.carry_end_location[0]
                players.add(event.player_id)
                last_carry_start_location = event.location
                last_carry_distance = math.dist(event.location, event.carry_end_location)
            elif event.type in ['Dispossessed', 'Miscontrol', 'Interception', 'Clearance', 'Block', 'Shot'] \
            or (event.type == 'Pass' and pd.notna(event.pass_outcome)) \
            or (event.type == 'Dribble' and event.dribble_outcome == 'Incomplete') \
            or (event.type == 'Foul Won' and pd.isna(event.foul_won_advantage)):
                break
            elif event.type == 'Dribble' and event.dribble_outcome == 'Complete':
                dribble = True
            elif dribble == True and event.type == 'Carry':
                if last_carry_distance <= 5.46807 and math.dist(last_carry_start_location, event.carry_end_location) > 5.46807:
                    event_count += 1
                final_location_x = event.carry_end_location[0]
                dribble = False
                last_carry_distance = math.dist(last_carry_start_location, event.carry_end_location)
            else:
                continue
    event_dict['successful_transition'] = True if final_location_x >= 80 else False
    event_dict['transition_final_location_x'] = final_location_x
    event_dict['transition_event_count'] = event_count 
    event_dict['transition_players_involved'] = len(players)
    data[receipt.id] = event_dict

In [7]:
results = pd.merge(
    left = ball_receipts,
    right = pd.DataFrame.from_dict(data=data, orient='index').reset_index().rename(columns = {'index': 'id'}),
    how = 'left',
    on = 'id'
)

results.head()

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,season,match_date,home_team,home_score,away_score,away_team,successful_transition,transition_final_location_x,transition_event_count,transition_players_involved
0,69e3b0a7-5847-40db-91d5-0e24fb150f63,3890561,9,0,2,2,175,Hoffenheim,Ball Receipt*,6039.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,False,34.3,1,2
1,d843ba4b-de1e-4e81-a71d-0f3781c8f3fa,3890561,63,0,56,3,181,Schalke 04,Ball Receipt*,3053.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,False,43.2,1,2
2,45271883-07e0-446f-ba83-3ddc3e678def,3890561,69,1,4,3,181,Schalke 04,Ball Receipt*,5242.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,False,74.7,3,2
3,7d08b692-1f2e-4b44-be64-7d3e04632ab9,3890561,86,1,14,3,181,Schalke 04,Ball Receipt*,5242.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,False,47.7,2,2
4,0a325d10-463e-4e70-86bd-1d2ae118864c,3890561,92,1,19,3,181,Schalke 04,Ball Receipt*,5242.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,False,46.8,1,2


In [8]:
results[(results.successful_transition == True)]

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,season,match_date,home_team,home_score,away_score,away_team,successful_transition,transition_final_location_x,transition_event_count,transition_players_involved
17,55d97f1e-4923-4f3b-b455-66a64594ed6d,3890561,369,7,28,13,181,Schalke 04,Ball Receipt*,6717.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,True,81.2,1,1
23,f85785ad-4ce6-4bd2-8694-4596a6c22d48,3890561,534,11,32,22,181,Schalke 04,Ball Receipt*,3499.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,True,104.2,2,2
27,0481012e-52f4-487a-98af-78cc8b82ace3,3890561,682,16,22,32,175,Hoffenheim,Ball Receipt*,6039.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,True,99.6,2,1
31,464ac823-fcb6-4f02-82cf-80d7220abdd0,3890561,732,17,1,33,181,Schalke 04,Ball Receipt*,8508.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,True,81.6,2,2
32,2ab487a3-79a4-4a3b-9c20-9f4e0734ffd4,3890561,735,17,1,33,181,Schalke 04,Ball Receipt*,3053.0,...,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04,True,81.6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,738ac391-8a02-4000-9a48-48b843473d51,3890261,2355,49,6,93,180,Borussia Dortmund,Ball Receipt*,5684.0,...,2015/2016,2015-08-15,Borussia Dortmund,4,0,Borussia Mönchengladbach,True,95.6,1,1
725,0f2b1d62-180e-41a0-a569-4ff36d2237b8,3890261,2436,51,25,96,180,Borussia Dortmund,Ball Receipt*,5684.0,...,2015/2016,2015-08-15,Borussia Dortmund,4,0,Borussia Mönchengladbach,True,110.5,2,1
756,97720d92-0a86-412a-9c77-2af59c660427,3890261,3521,74,27,140,180,Borussia Dortmund,Ball Receipt*,8816.0,...,2015/2016,2015-08-15,Borussia Dortmund,4,0,Borussia Mönchengladbach,True,84.9,3,2
773,159f86de-6227-4314-ad49-27d9acb2d96d,3890261,3985,85,52,166,180,Borussia Dortmund,Ball Receipt*,8816.0,...,2015/2016,2015-08-15,Borussia Dortmund,4,0,Borussia Mönchengladbach,True,82.3,2,2


In [30]:
df = results[['id', 'player_id', 'player', 'position', 'team_id', 'team', 'under_pressure', 'successful_transition', 'transition_players_involved']]

df.head()

,id,player_id,player,position,team_id,team,under_pressure,successful_transition,transition_players_involved
0,69e3b0a7-5847-40db-91d5-0e24fb150f63,6039.0,Sebastian Rudy,Left Defensive Midfield,175,Hoffenheim,NaN,False,2
1,d843ba4b-de1e-4e81-a71d-0f3781c8f3fa,3053.0,Leroy Sané,Right Midfield,181,Schalke 04,NaN,False,2
2,45271883-07e0-446f-ba83-3ddc3e678def,5242.0,Younès Belhanda,Left Center Midfield,181,Schalke 04,NaN,False,2
3,7d08b692-1f2e-4b44-be64-7d3e04632ab9,5242.0,Younès Belhanda,Left Center Midfield,181,Schalke 04,NaN,False,2
4,0a325d10-463e-4e70-86bd-1d2ae118864c,5242.0,Younès Belhanda,Left Center Midfield,181,Schalke 04,NaN,False,2


In [45]:
sqldf(
    '''
    SELECT 
        CAST(player_id AS INT) AS player_id,
        player,
        team_id,
        team,
        COUNT(id) AS total_receptions,
        COUNT(CASE WHEN successful_transition = TRUE THEN id END) AS total_successful_transitions,
        ROUND(COUNT(CASE WHEN successful_transition = TRUE THEN id END) * 100.0 / COUNT(id), 2) AS successful_transition_pct
    FROM df
    GROUP BY 1, 2, 3, 4
    ORDER BY 7 DESC
    '''
)

,player_id,player,team_id,team,total_receptions,total_successful_transitions,successful_transition_pct
0,5208,Thiago Alcântara do Nascimento,169,Bayern Munich,12,6,50.00
1,11281,Alessandro Schöpf,181,Schalke 04,4,2,50.00
2,5562,Thomas Müller,169,Bayern Munich,5,2,40.00
3,8388,Jérôme Gondorf,872,Darmstadt 98,7,2,28.57
4,5684,Shinji Kagawa,180,Borussia Dortmund,8,2,25.00
5,8816,Julian Weigl,180,Borussia Dortmund,22,5,22.73
6,10287,İlkay Gündoğan,180,Borussia Dortmund,16,3,18.75
7,8543,Manuel Schmiedebach,178,Hannover 96,11,2,18.18
8,8578,Per Ciljan Skjelbred,173,Hertha Berlin,34,6,17.65
9,8404,Eugen Polanski,175,Hoffenheim,6,1,16.67
